In [4]:
import os 
from pathlib import Path
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt

# Get short-version beer data (maximum length 120)

In [5]:
aspect=['Look','Aroma','Palate']
mode=['train','dev','test']

len_text_look=0
len_text_aroma=0
len_text_palate=0
for (aspect, mode) in list(itertools.product(aspect,mode)):
    if mode in ('train', 'dev'):
        path =  f'/home/ec2-user/SageMaker/rationale-causal/data/sentiment/data/source/beer_{aspect}.{mode}'
    elif mode == 'test':
        path =  f'/home/ec2-user/SageMaker/rationale-causal/data/sentiment/data/target/beer_{aspect}.train'
    df = pd.read_csv(path, delimiter='\t')
    indexs=[]
    labels=[]
    for index, row in df.iterrows():
            text = row['text'].split()
            if len(text)<=120 or mode=='test':
                label = row['label']
                # this could be applied to both beer and hotel
                if label >= 0.6:
                    label = 1  # pos
                    labels.append(label) 
                    indexs.append(index)
                    if aspect=='Look':
                        len_text_look+=len(text)
                    elif aspect=='Aroma':
                        len_text_aroma+=len(text)
                    elif aspect=='Palate':
                        len_text_palate+=len(text)
                elif label <= 0.4:
                    label = 0  # neg
                    labels.append(label)
                    indexs.append(index)
                    if aspect=='Look':
                        len_text_look+=len(text)
                    elif aspect=='Aroma':
                        len_text_aroma+=len(text)
                    elif aspect=='Palate':
                        len_text_palate+=len(text)
                else:
                    continue   


    
    df=df.iloc[indexs]
    df['labels']=labels
    globals()['beer_'+aspect+'_'+mode]=df
    print(f'beer {aspect} {mode}: number of short samples:{len(df.index)}')
    print(f'beer {aspect} {mode}: number of positive samples:{sum(labels)}, number of negative samples:{len(labels)-sum(labels)}, total:{len(labels)}')

    


beer Look train: number of short samples:15932
beer Look train: number of positive samples:11807, number of negative samples:4125, total:15932
beer Look dev: number of short samples:3757
beer Look dev: number of positive samples:2794, number of negative samples:963, total:3757
beer Look test: number of short samples:200
beer Look test: number of positive samples:100, number of negative samples:100, total:200
beer Aroma train: number of short samples:14085
beer Aroma train: number of positive samples:10317, number of negative samples:3768, total:14085
beer Aroma dev: number of short samples:2928
beer Aroma dev: number of positive samples:2168, number of negative samples:760, total:2928
beer Aroma test: number of short samples:200
beer Aroma test: number of positive samples:100, number of negative samples:100, total:200
beer Palate train: number of short samples:9592
beer Palate train: number of positive samples:7990, number of negative samples:1602, total:9592
beer Palate dev: number of

# Get noise-version beer data (add spurious correlation) 

In [6]:
def add_noise(raw_data,alpha,mode):
    data=raw_data.copy()
    labels=data['labels'].to_numpy()
    if mode=='test':
        alpha=1-alpha
    else:
        alpha=alpha
    for i in range(len(data.index)):
        indi=np.random.binomial(1,alpha,1)
        data.loc[data.index[i],'rationale']='0 '+data.loc[data.index[i],'rationale']
        if indi==1:
            if labels[i]==1:
                    data.loc[data.index[i],'text']=', '+data.loc[data.index[i],'text']
            else:
                    data.loc[data.index[i],'text']='. '+data.loc[data.index[i],'text']
        else:
            if labels[i]==1:
                    data.loc[data.index[i],'text']='. '+data.loc[data.index[i],'text']
            else:
                    data.loc[data.index[i],'text']=', '+data.loc[data.index[i],'text']
    return data



In [7]:
alpha=0.8
aspect=['Aroma','Palate']
mode=['train','dev']
np.random.seed(0)
for (aspect, mode) in list(itertools.product(aspect,mode)):
    df=(globals()['beer_'+aspect+'_'+mode])
    globals()['beer_'+aspect+'_'+mode+'_noise']=add_noise(df,alpha,mode)
    if mode in ('train', 'dev'):
        path =  f'/home/ec2-user/SageMaker/Causal-Rationale/rationale-causal/data/sentiment/data/source/beer_{aspect}.{mode}'
    globals()['beer_'+aspect+'_'+mode+'_noise'].to_csv(path+'_noise',encoding='utf-8',sep='\t',index=False)